In [ ]:
import numpy as np
import probnum as pn
import linpde_gp

from matplotlib import pyplot as plt

np.random.seed(932401)

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

figures_path = Path(os.environ.get("FIGURES_DIR"))

In [ ]:
def true_fn(x):
    return np.sin(x)

def true_fn_deriv(x):
    return np.cos(x)

domain = linpde_gp.domains.Interval(0, 2 * np.pi)

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
        mean=linpde_gp.functions.Zero(input_shape=()),
        cov=linpde_gp.randprocs.covfuncs.Matern((), nu=2.5, lengthscales=np.pi/2),
    )

In [ ]:
dx = linpde_gp.linfuncops.diffops.Derivative(1)

In [ ]:
u_bc = u_prior.condition_on_observations(X=domain.uniform_grid((2,)), Y=np.zeros((2,)), noise=1e-8)

In [ ]:
N_eval = 50
X_eval = domain.uniform_grid(N_eval)

global_noise = np.random.randn(N_eval, 3)

def plot_with_samples(gp, true_fn_label=None, fig=None, ax=None, noise=global_noise, gp_label="GP"):
    if fig is None or ax is None:
        fig, ax = plt.subplots()
    gp_label = r'\nicefrac{d}{dx}(' + gp_label + ")" if true_fn_label == "derivative" else gp_label
    means = gp.mean(X_eval)
    stds = np.sqrt(gp.cov.linop(X_eval).diagonal())
    ax.plot(X_eval, means, label=gp_label)
    ax.fill_between(X_eval, means - 1.96 * stds, means, color="cornflowerblue", alpha=0.3)
    ax.fill_between(X_eval, means + 1.96 * stds, means, color="cornflowerblue", alpha=0.3)

    # Samples
    cov_mat = gp.cov.linop(X_eval)
    samples = np.linalg.cholesky(cov_mat.todense()) @ noise
    samples = means + samples.T
    for i in range(samples.shape[0]):
        ax.plot(X_eval, samples[i], color="gray", alpha=0.4)
    
    if true_fn_label == "original":
        ax.plot(X_eval, true_fn(X_eval), color="gold", label="True", linestyle="dashed")
    
    if true_fn_label == "derivative":
        ax.plot(X_eval, true_fn_deriv(X_eval), color="gold", label=r'\nicefrac{d}{dx}(True)', linestyle="dashed")
    
    ax.legend()

In [ ]:
plot_with_samples(u_bc, true_fn_label="original")

In [ ]:
X_col = domain.uniform_grid((4,), centered=True)
u_col = u_bc.condition_on_observations(X=X_col, Y=true_fn_deriv(X_col), L=dx)

In [ ]:
X_col

In [ ]:
plot_with_samples(u_col)

In [ ]:
from linpde_gp.domains import TensorProductDomain, VectorizedDomain

domains = TensorProductDomain.from_endpoints(np.linspace(*domain, 4+1))
domains_int = [d[0] for d in domains]
domains = VectorizedDomain(domains_int)
L_fv = linpde_gp.linfunctls.FiniteVolumeFunctional(domains, dx)

In [ ]:
Y_fv = np.array([np.sin(d[1]) - np.sin(d[0]) for d in domains_int])
Y_fv

In [ ]:
u_fv = u_bc.condition_on_observations(L=L_fv, Y=Y_fv)

In [ ]:
plot_with_samples(u_fv)

In [ ]:
plot_with_samples(dx(u_col), true_fn_label="derivative")

In [ ]:
plot_with_samples(dx(u_fv), true_fn_label="derivative")

In [ ]:
from tueplots import bundles
plt.rcParams.update(bundles.neurips2024(nrows=2, ncols=2))
plt.rc("text.latex", preamble=r'\usepackage{nicefrac}')
# plt.rcParams.update(bundles.beamer_moml())

# plt.rcParams.clear()

fig, ax = plt.subplots(2, 2)

In [ ]:
plot_with_samples(u_col, "original", fig, ax[0, 0])
plot_with_samples(dx(u_col), "derivative", fig, ax[1, 0])
plot_with_samples(u_fv, "original", fig, ax[0, 1])
plot_with_samples(dx(u_fv), "derivative", fig, ax[1, 1])

In [ ]:
import matplotlib.ticker as tck

ax[1, 0].plot(X_col, true_fn_deriv(X_col), "x", color="red", alpha=0.6, markersize=8, markeredgewidth=2)
plt.rcParams.update({"text.usetex": True})

for i in range(len(Y_fv)):
    y = Y_fv[i] / (np.pi / 2)
    xstart = domains_int[i][0]
    xstop = domains_int[i][1]
    ax[1, 1].hlines(y, xstart, xstop, "red", lw=2, alpha=0.6)
    ax[1, 1].vlines(xstart, y+0.2, y-0.2, "red", lw=2, alpha=0.6)
    ax[1, 1].vlines(xstop, y+0.2, y-0.2, "red", lw=2, alpha=0.6)
    ax[1, 1].plot(Y_fv[i], domains_int[i][0], domains_int[i][1])

for i in [0, 1]:
    for j in [0, 1]:
        ax[i, j].set_xlim(0.0 - 0.1, 2*np.pi + 0.1)
        ax[i, j].set_ylim(-2.0 - 0.2, 2.0)
        ax[i, j].xaxis.set_major_locator(tck.FixedLocator(np.linspace(0.0, 2 * np.pi, 4+1)))
        ax[i, j].set_xticklabels(["0", "$\\nicefrac{\\pi}{2}$", "$\\pi$", "$\\nicefrac{3 \\pi}{2}$", "$2 \\pi$"])

ax[0, 0].set_title("Collocation")
ax[0, 1].set_title("FVM")

ax[0, 0].set_ylabel("$u(x)$")
ax[1, 0].set_ylabel(r'$\nicefrac{d u}{dx}(x)$')
ax[1, 0].set_xlabel("$x$")
ax[1, 1].set_xlabel("$x$")

In [ ]:
fig

In [ ]:
fig.savefig(figures_path / "collocation_vs_fvm.pdf")